<a href="https://colab.research.google.com/github/DikshantBadawadagi/Encryption-Algorithms/blob/main/Arnold-Transformation-Encryption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
from PIL import Image
import random

def logistic_map_2d(x0, y0, r, shape):
    x, y = x0, y0
    sequence = np.zeros(shape)
    for i in range(shape[0]):
        for j in range(shape[1]):
            x = r * x * (1 - x)
            y = r * y * (1 - y)
            sequence[i, j] = (x + y) / 2
    return sequence

def image_to_array(image_path):
    image = Image.open(image_path)
    return np.array(image)

def array_to_image(array, output_path):
    array = np.clip(array, 0, 255).astype(np.uint8)
    image = Image.fromarray(array)
    image.save(output_path)

def arnold_transform(image, num_iterations):
    height, width = image.shape[:2]
    result = np.zeros_like(image)

    for _ in range(num_iterations):
        for y in range(height):
            for x in range(width):
                new_x = (x + y) % width
                new_y = (x + 2*y) % height
                result[new_y, new_x] = image[y, x]
        image = result.copy()

    return result

def inverse_arnold_transform(image, num_iterations):
    height, width = image.shape[:2]
    result = np.zeros_like(image)

    for _ in range(num_iterations):
        for y in range(height):
            for x in range(width):
                new_x = (2*x - y) % width
                new_y = (-x + y) % height
                result[new_y, new_x] = image[y, x]
        image = result.copy()

    return result

def encrypt_image(image_array, key, arnold_iterations):
    encrypted = arnold_transform(image_array, arnold_iterations)
    encrypted = encrypted.astype(np.int16)
    if len(image_array.shape) == 3:  # Color image
        for c in range(image_array.shape[2]):
            encrypted[:,:,c] = (encrypted[:,:,c] + (key * 256).astype(np.int16)) % 256
    else:  # Grayscale image
        encrypted = (encrypted + (key * 256).astype(np.int16)) % 256
    return encrypted.astype(np.uint8)

def decrypt_image(encrypted_array, key, arnold_iterations):
    decrypted = encrypted_array.astype(np.int16)
    if len(encrypted_array.shape) == 3:  # Color image
        for c in range(encrypted_array.shape[2]):
            decrypted[:,:,c] = (decrypted[:,:,c] - (key * 256).astype(np.int16)) % 256
    else:  # Grayscale image
        decrypted = (decrypted - (key * 256).astype(np.int16)) % 256
    decrypted = inverse_arnold_transform(decrypted.astype(np.uint8), arnold_iterations)
    return decrypted.astype(np.uint8)

def encrypt_image_file(input_path, output_path):
    x0 = random.uniform(0, 1)
    y0 = random.uniform(0, 1)
    r = random.uniform(3.8, 4.0)
    arnold_iterations = random.randint(1, 10)
    image_array = image_to_array(input_path)
    key = logistic_map_2d(x0, y0, r, image_array.shape[:2])
    encrypted_array = encrypt_image(image_array, key, arnold_iterations)
    array_to_image(encrypted_array, output_path)
    return x0, y0, r, arnold_iterations

def decrypt_image_file(encrypted_path, output_path, x0, y0, r, arnold_iterations):
    encrypted_array = image_to_array(encrypted_path)
    key = logistic_map_2d(x0, y0, r, encrypted_array.shape[:2])
    decrypted_array = decrypt_image(encrypted_array, key, arnold_iterations)
    array_to_image(decrypted_array, output_path)

# Example Usage
input_image_path = '/content/download.jpg'
encrypted_image_path = '/content/encrypted_image.png'
decrypted_image_path = '/content/decrypted_image.png'

# Encryption
x0, y0, r, arnold_iterations = encrypt_image_file(input_image_path, encrypted_image_path)
print(f"Encryption parameters: x0={x0}, y0={y0}, r={r}, arnold_iterations={arnold_iterations}")

# Decryption
decrypt_image_file(encrypted_image_path, decrypted_image_path, x0, y0, r, arnold_iterations)
print("Decryption complete.")

# Debugging
original_image = image_to_array(input_image_path)
encrypted_image = image_to_array(encrypted_image_path)
decrypted_image = image_to_array(decrypted_image_path)

print("Original image shape:", original_image.shape)
print("Encrypted image shape:", encrypted_image.shape)
print("Decrypted image shape:", decrypted_image.shape)

print("Original image dtype:", original_image.dtype)
print("Encrypted image dtype:", encrypted_image.dtype)
print("Decrypted image dtype:", decrypted_image.dtype)

print("Original image sample:", original_image[0, 0])
print("Encrypted image sample:", encrypted_image[0, 0])
print("Decrypted image sample:", decrypted_image[0, 0])

difference = np.abs(original_image.astype(np.float64) - decrypted_image.astype(np.float64))
mean_difference = np.mean(difference)
print(f"Mean difference between original and decrypted images: {mean_difference}")

max_difference = np.max(difference)
print(f"Maximum difference between original and decrypted images: {max_difference}")

# Check if any pixel values are different
if np.array_equal(original_image, decrypted_image):
    print("The decrypted image is identical to the original image.")
else:
    print("The decrypted image differs from the original image.")
    different_pixels = np.sum(original_image != decrypted_image)
    print(f"Number of different pixels: {different_pixels}")

Encryption parameters: x0=0.6113058620999394, y0=0.7963517272202025, r=3.840622762009501, arnold_iterations=4
Decryption complete.
Original image shape: (225, 225, 3)
Encrypted image shape: (225, 225, 3)
Decrypted image shape: (225, 225, 3)
Original image dtype: uint8
Encrypted image dtype: uint8
Decrypted image dtype: uint8
Original image sample: [255 255 255]
Encrypted image sample: [195 195 195]
Decrypted image sample: [255 255 255]
Mean difference between original and decrypted images: 0.0
Maximum difference between original and decrypted images: 0.0
The decrypted image is identical to the original image.
